# Prepare data

First the data is copied from the google drive into colab for faster processing time

In [1]:
!cp -r "/content/drive/MyDrive/Colab Notebooks/project/tif_images.tar" /content/

In [2]:
!tar -xf /content/tif_images.tar -C /content

In [3]:
# delete old labels from first test runs
!rm -rf "/content/tif_images/train/labels"
!rm -rf "/content/tif_images/val/labels"
!rm -rf "/content/tif_images/val/labels.cache"
!rm -rf "/content/tif_images/train/labels.cache"

Quick check whether number of iamges and labels are the same

In [4]:
# install ultrlytics
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.2/902.2 kB 57.0 MB/s eta 0:00:00


In [13]:
# all imports
from pathlib import Path
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
import os
import shutil
import ultralytics
from ultralytics import YOLO

In [ ]:


def count_files_in_folder(folder_path):
    """
    Counts the number of files in a given folder using pathlib.

    Parameters:
    folder_path (str): Path to the folder.

    Returns:
    int: Number of files in the folder.
    """
    path = Path(folder_path)

    if not path.exists():
        raise FileNotFoundError(f"The folder '{folder_path}' does not exist.")

    if not path.is_dir():
        raise NotADirectoryError(f"'{folder_path}' is not a directory.")

    # Use Path.iterdir() for faster iteration
    return sum(1 for item in path.iterdir() if item.is_file())

In [ ]:
folder = Path(r"/content/tif_images")

In [ ]:
train_im = folder / "train" / "images"
val_im = folder / "val" / "images"
train_labels = folder/ "labels_general_train"
val_labels = folder / "labels_general_val"
print(count_files_in_folder(train_labels))
print(count_files_in_folder(val_labels))
print(count_files_in_folder(train_im))
print(count_files_in_folder(val_im))

27299
6830
27299
6830


## Create labels
From labels with 60 classes, simplified labels with 1 class only is created

In [ ]:
# Reminder how the labels look like

parent_labels_dict = {
    -1: "None",
    0: "Fixed-wing Aircraft",
    1: "Passenger Vehicle",
    2: "Truck",
    3: "Railway Vehicle",
    4: "Maritime Vessel",
    5: "Engineering Vehicle",
    6: "Building"
}

labels_dict = {
    0: "Fixed-wing Aircraft",
    1: "Small Aircraft",
    2: "Cargo Plane",
    3: "Helicopter",
    4: "Passenger Vehicle",
    5: "Small Car",
    6: "Bus",
    7: "Pickup Truck",
    8: "Utility Truck",
    9: "Truck",
    10: "Cargo Truck",
    11: "Truck w/Box",
    12: "Truck Tractor",
    13: "Trailer",
    14: "Truck w/Flatbed",
    15: "Truck w/Liquid",
    16: "Crane Truck",
    17: "Railway Vehicle",
    18: "Passenger Car",
    19: "Cargo Car",
    20: "Flat Car",
    21: "Tank car",
    22: "Locomotive",
    23: "Maritime Vessel",
    24: "Motorboat",
    25: "Sailboat",
    26: "Tugboat",
    27: "Barge",
    28: "Fishing Vessel",
    29: "Ferry",
    30: "Yacht",
    31: "Container Ship",
    32: "Oil Tanker",
    33: "Engineering Vehicle",
    34: "Tower crane",
    35: "Container Crane",
    36: "Reach Stacker",
    37: "Straddle Carrier",
    38: "Mobile Crane",
    39: "Dump Truck",
    40: "Haul Truck",
    41: "Scraper/Tractor",
    42: "Front loader/Bulldozer",
    43: "Excavator",
    44: "Cement Mixer",
    45: "Ground Grader",
    46: "Hut/Tent",
    47: "Shed",
    48: "Building",
    49: "Aircraft Hangar",
    50: "Damaged Building",
    51: "Facility",
    52: "Construction Site",
    53: "Vehicle Lot",
    54: "Helipad",
    55: "Storage Tank",
    56: "Shipping container lot",
    57: "Shipping Container",
    58: "Pylon",
    59: "Tower"
}

In [6]:
def convert_class_road_vehicle(child_class):
  if 4 <= child_class <= 16:
    return 0
  else:
    return -1

def convert_class_building(child_class):
  if child_class == 48:
    return 0
  else:
    return -1

def convert_class_car(child_class):
  if child_class == 5:
    return 0
  else:
    return -1


# Source and target directories
source_train_labels = "/content/tif_images/labels_general_train"
source_val_labels = "/content/tif_images/labels_general_val"
target_train_labels = "/content/tif_images/train/labels"
target_val_labels = "/content/tif_images/val/labels"

# Ensure target directories exist
os.makedirs(target_train_labels, exist_ok=True)
os.makedirs(target_val_labels, exist_ok=True)

def process_labels(source_folder, target_folder, convert_function):
    """
    Process labels by applying class mapping and copying to the target folder.
    """
    for label_file in tqdm(os.listdir(source_folder)):
        if label_file.endswith(".txt"):
            source_label_path = os.path.join(source_folder, label_file)
            target_label_path = os.path.join(target_folder, label_file)

            # Read the label file
            with open(source_label_path, "r") as f:
                lines = f.readlines()

            # Process the label lines with the class mapping
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                class_id = int(float(parts[0]))  # Original class ID
                new_class_id = convert_function(class_id)
                if new_class_id == -1:
                    continue
                parts[0] = str(new_class_id)
                new_lines.append(" ".join(parts))


            # If any labels are valid, save them to the target folder
            if new_lines:
                with open(target_label_path, "w") as f:
                    f.write("\n".join(new_lines) + "\n")
            else:
                # If no valid labels, create an empty label file
                with open(target_label_path, "w") as f:
                    pass  # Empty file

# Process train and val labels
process_labels(source_train_labels, target_train_labels, convert_class_car)
process_labels(source_val_labels, target_val_labels, convert_class_car)

print("Labels processed and saved successfully!")

100%|██████████| 6830/6830 [00:00<00:00, 10045.53it/s]

Labels processed and saved successfully!


In [7]:
# prepare config
config = """
# train and val datasets (image directory or *.txt file with image paths)
train: /content/tif_images/train
val: /content/tif_images/val

# number of classes
nc: 1

# class names
names: ['Car']
"""

with open("/content/data.yaml", "w") as f:
    f.write(config)

In [12]:
checkpoint_path = "/content/last.pt"

try:
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    print("Checkpoint keys: ", checkpoint.keys())
except Exception as e:
    print(f"Error loading checkpoint: {e}")

Error loading checkpoint: PytorchStreamReader failed reading zip archive: failed finding central directory


In [14]:
root = "/content"
# from scratch
!yolo task=detect mode=train model=yolov8m.pt data={root}/data.yaml epochs=30 imgsz=512 mosaic=0.0 flipud=0.5 scale=0.0
# from pretrained model
# !yolo task=detect mode=train model={root}/last.pt data={root}/data.yaml epochs=20 imgsz=512 mosaic=0.0 flipud=0.5 scale=0.0
model = YOLO(f'{root}/last.pt')
model.train(resume=True, epochs=20)

100% 49.7M/49.7M [00:00<00:00, 223MB/s]
Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

train: Scanning /content/tif_images/train/labels.cache... 27299 images, 18627 backgrounds, 0 corrupt: 100%|██████████| 27299/27299 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/tif_images/train/images/1529_448_896.tif: 1 duplicate labels removed
train: WARNING ⚠️ /content/tif_images/train/images/18_0_0.tif: 1 duplicate labels removed
train: WARNING ⚠️ /content/tif_images/train/images/217_2688_2240.tif: 1 duplicate labels removed
train: WARNING ⚠️ /content/tif_images/train/images/252_3136_1792.tif: 1 duplicate labels removed
train: WARNING ⚠️ /content/tif_images/train/images/562_0_448.tif: 1 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/tif_images/val/labels.cache... 6830 images, 4326 backgrounds, 0 corrupt: 100%|██████████| 6830/6830 [00:00<?, ?it/s]

...

      20/20      10.7G      2.056      1.094     0.9035          0        512: 100%|██████████| 1707/1707 [02:59<00:00,  9.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 214/214 [00:21<00:00,  9.73it/s]


                   all       6830      55205      0.754      0.771      0.763      0.305

13 epochs completed in 0.737 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 214/214 [00:28<00:00,  7.52it/s]


                   all       6830      55205      0.754       0.77      0.763      0.305
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7992dae8b760>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
# download the files

In [15]:
import shutil

# Zip the folder
shutil.make_archive('runs', 'zip', 'runs')

from google.colab import files

# Download the zip file
files.download('runs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
!yolo task=detect mode=train model=/content/runs/detect/train/weights/last.pt data={root}/data.yaml epochs=10 imgsz=512 mosaic=0.0 flipud=0.5 scale=0.0

Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train/weights/last.pt, data=/content/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

In [19]:
!yolo task=detect mode=train model=/content/runs/detect/train4/weights/last.pt data={root}/data.yaml epochs=10 imgsz=512 mosaic=0.0 flipud=0.5 scale=0.0

Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train4/weights/last.pt, data=/content/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_